In [1]:
import pandas as pd
import datetime, pytz
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import json

import talib
import pandas_ta
import time

from importlib import reload


import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_strategies import Strategy as Strat

c:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [3]:
def alligator(Close, window1, window2, window3, window4, window5, window6):
    ema1 = vbt.talib('EMA').run(Close, window1).real.to_numpy()
    ema2 = vbt.talib('EMA').run(Close, window2).real.to_numpy()
    ema3 = vbt.talib('EMA').run(Close, window3).real.to_numpy()
    ema4 = vbt.talib('EMA').run(Close, window4).real.to_numpy()
    ema5 = vbt.talib('EMA').run(Close, window5).real.to_numpy()
    ema6 = vbt.talib('EMA').run(Close, window6).real.to_numpy()
    stoch_rsi = vbt.talib('STOCHRSI').run(Close).fastk.to_numpy() 

    entries = (ema1 > ema2) & (ema2 > ema3) & (ema3 > ema4) & (ema4 > ema5) & (ema5 > ema6) & (stoch_rsi < 0.8)
    exits = (ema6 > ema1) & (stoch_rsi < 0.2) 

    return entries, exits

Alligator = vbt.IndicatorFactory(
            class_name= 'Alligator',
            short_name = 'ALGT',
            input_names = ["Close"] ,
            param_names = ["window1", "window2", "window3", "window4", "window5", "window6",],
            output_names= ["entries", "exits"]
        ).from_apply_func(alligator)

In [ ]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 8  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',            
        },
        'get_params': {
            'column': ['High', 'Low', 'Close']
        },
        'run_params': { 
            'window1': {'start': 5, 'stop': 10, 'step': 3},
            'window2': {'start': 10, 'stop': 15, 'step': 3},
            'window3': {'start': 15, 'stop': 25, 'step': 3},
            'window4': {'start': 30, 'stop': 40, 'step': 3},
            'window5': {'start': 50, 'stop': 60, 'step': 3},
            'window6': {'start': 100, 'stop': 110, 'step': 5},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [2]:
strat = Strat()
strat.TrueStrat

vectorbt.indicators.factory.TrueStrat

In [5]:

test = ForwardTest(Alligator, params)
test.backtest(init_cash=1000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-11-26 00:00:00+00:00 and 2017-12-10 00:00:00+00:00 finished at 2023-07-17 16:18:15.041983
Cash: 1028.1426649211196


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

Forward test between 2018-02-04 00:00:00+00:00 and 2018-02-18 00:00:00+00:00 finished at 2023-07-17 16:19:52.911802
Cash: 1068.2732427405765


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2018-02-18 00:00:00+00:00 and 2018-03-04 00:00:00+00:00 finished at 2023-07-17 16:20:11.199082
Cash: 1076.0054902237362


0it [00:00, ?it/s]

In [ ]:
pf = test.aggregate_results ()
pf.stats()